Exercise 3: Little Lemon analysis and sales report

In [1]:
"""
To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.
To find the information on the error, import the Error class from MySQL Connector/Python.
"""

from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import mysql.connector as connector
import pandas as pd

Task 1:
Complete the following steps to establish a connection pool:

To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.

To find the information on the error, import the Error class from MySQL Connector/Python.

Define your database configurations as a Python dictionary object called dbconfig.

Establish a connection pool [pool_name = pool_b] with two connections. 

Implement error handling using a try-except block in case the connection fails.

In [2]:
"""
Define your database configurations as a Python dictionary object called dbconfig.
"""

dbconfig = {
    'user' : 'root',
    'password' : '******',
    'database' : 'meta_pytomysql_test'
}

In [3]:
"""
Establish a connection pool [pool_name = pool_b] with two connections. 
Implement error handling using a try-except block in case the connection fails.
"""

try:
    pool = MySQLConnectionPool(
        pool_name = 'pool_b',
        pool_size =  2,
        **dbconfig
    )
    print('The connection pool is created with a name: ', pool.pool_name)
    print('The pool size is: ', pool.pool_size)

except Error as err:
    print('Error code: ', err.errno)
    print('Error message: ', err.msg)

The connection pool is created with a name:  pool_b
The pool size is:  2


Task 2:
Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:

TIP: You need to add a connection to connect the third guest.

Guest 1:
Table Number: 8
First Name: Anees
Last Name: Java
Booking Time: 18:00
EmployeeID: 6

Guest 2:
Table Number: 5
First Name: Bald
Last Name: Vin
Booking Time: 19:00
EmployeeID: 6

Guest 3:
Table Number: 12
First Name: Jay
Last Name: Kon
Booking Time: 19:30 
EmployeeID: 6

Return all the connections back to the pool. 

TIP: The pool size is two. However, you have three connected users. You can only return two connections. Returning a third connection will raise a PoolError. Use try-except to print the error message.  

In [4]:
# Task 2:

# Connect the guest 1.
connection1 = pool.get_connection()
cursor1=connection1.cursor()
query_booking1="""INSERT INTO bookings 
(tableNo, guestFirstName, guestLastName, bookingSlot, employeeID)
VALUES (8,'Anees','Java','18:00:00',6);"""
cursor1.execute(query_booking1)
connection1.commit()
print("The booking relative to the guest 1 is added into the table")

# Connect the guest 2.
connection2 = pool.get_connection()
cursor2=connection2.cursor()
query_booking2="""INSERT INTO bookings 
(tableNo, guestFirstName, guestLastName, bookingSlot, employeeID)
VALUES (5, 'Bald','Vin','19:00:00',6);"""
cursor2.execute(query_booking2)
connection2.commit()
print("The booking relative to the guest 2 is added into the table")

# Adding a new connection to connect the guest 3.

try:
    connection3 = pool.get_connection()
    print("The guest 3 is in the pool")
except:
    print("Adding a new connection to the pool...")
        
    # Create a connection
    connection=connector.connect(**dbconfig)
    # Add the connection into the pool
    pool.add_connection(cnx=connection)
    print("A new connection is added in the pool.\n")
        
    connection3 = pool.get_connection()
    print("'connection3' is added in the pool.")

# Create the cursor and insert query of the guest 3: 
cursor3 = connection3.cursor()
query_booking3="""INSERT INTO bookings 
(tableNo, guestFirstName, guestLastName, bookingSlot, employeeID)
VALUES (12, 'Jay','Kon','19:30:00',6);"""
cursor3.execute(query_booking3)
connection3.commit()
print("The booking relative to the guest 3 is added into the table")

The booking relative to the guest 1 is added into the table
The booking relative to the guest 2 is added into the table
Adding a new connection to the pool...
A new connection is added in the pool.

'connection3' is added in the pool.
The booking relative to the guest 3 is added into the table


In [6]:
# Return all the connections back to the pool. 

for connection in [connection1,connection2, connection3]:
    try:
        connection.close()
        print("Connection is returned to the pool")
    except Error as er:
        print("\nConnection can't be returned to the pool" )
        print("Error message:", er.msg)

Connection is returned to the pool
Connection is returned to the pool

Connection can't be returned to the pool
Error message: Failed adding connection; queue is full


Task 3:
Create a report containing the following information:

The name and EmployeeID of the Little Lemon manager.

The name and role of the employee who receives the highest salary.

The number of guests booked between 18:00 and 20:00.

The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [7]:
# Task 3:
connection = pool.get_connection()
print("The object 'connection' is created with a connection link from the pool_b")

try:
    cursor = connection.cursor()
    print('cursor created.')
except:
    print("Can't create the cursor")

The object 'connection' is created with a connection link from the pool_b
cursor created.


In [8]:
# Task 3.1: The name and EmployeeID of the Little Lemon manager.

query_report1 = """SELECT name AS 'Name', EmployeeID FROM employees WHERE role = 'Manager';"""
cursor.execute(query_report1)

results1 = cursor.fetchall()
columns1 = cursor.column_names

df1 = pd.DataFrame(data=results1, columns=columns1)
df1.style.hide(axis='index')

Name,EmployeeID
Mario Gollini,1


In [9]:
# Task 3.2: The name and role of the employee who receives the highest salary.

query_report2 = """SELECT name AS 'Name', role AS 'Role' FROM employees ORDER BY annual_salary DESC LIMIT 1;"""
cursor.execute(query_report2)

results2 = cursor.fetchall()
columns2 = cursor.column_names

df2 = pd.DataFrame(data=results2, columns=columns2)
df2.style.hide(axis='index')

Name,Role
Mario Gollini,Manager


In [10]:
# Task 3.3: The number of guests booked between 18:00 and 20:00.

query_report3 = """SELECT COUNT(*) AS 'Bookings: 18:00 - 20:00' FROM bookings WHERE bookingSlot BETWEEN '18:00' AND '20:00';"""
cursor.execute(query_report3)

results3 = cursor.fetchall()
columns3 = cursor.column_names

df3 = pd.DataFrame(data=results3, columns=columns3)
df3.style.hide(axis='index')

Bookings: 18:00 - 20:00
7


In [11]:
# Task 3.4: The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

query_report4 = """SELECT 
bookingID, 
CONCAT(guestFirstName, ' ',guestLastName) AS 'Guest Name',
bookingSlot,
'Receptionist' AS 'To Employee'
FROM bookings WHERE employeeID = (SELECT employeeID FROM employees WHERE role = 'Receptionist')
ORDER BY bookingSlot;"""
cursor.execute(query_report4)

results4 = cursor.fetchall()
columns4 = cursor.column_names

df4 = pd.DataFrame(data=results4, columns=columns4)
df4.style.hide(axis='index')

bookingID,Guest Name,bookingSlot,To Employee
7,Anees Java,0 days 18:00:00,Receptionist
8,Bald Vin,0 days 19:00:00,Receptionist
9,Jay Kon,0 days 19:30:00,Receptionist


Task 4:
Create a stored procedure named BasicSalesReport that returns the following statistics: 

Total sales
Average sale
Minimum bill paid
Maximum bill paid

In [12]:
# Task 4: CREATE THE PROCEDURE
cursor1.execute("""DROP PROCEDURE IF EXISTS BasicSalesReport;""")

procedure_BasicSalesReport = """
CREATE PROCEDURE BasicSalesReport()
BEGIN

SELECT SUM(billAmount) 'Total sales',
AVG(billAmount) 'Average sale',
MIN(billAmount) 'Minimum bill paid',
MAX(billAmount) 'Maximum bill paid' 
FROM orders;

END
"""
cursor1.execute(procedure_BasicSalesReport)
print('Procedure created.')

Procedure created.


In [13]:
# Task 4.1: Call the procedure BasicSalesReport.

cursor1.callproc('BasicSalesReport')
results5 = next(cursor1.stored_results())
dataset5 = results5.fetchall()
columns5 = results5.column_names

df5 = pd.DataFrame(data=dataset5, columns=columns5)
df5.style.hide(axis='index')

Total sales,Average sale,Minimum bill paid,Maximum bill paid
243,48.6000,37,86


Task 5:
Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. To complete this task, carry out the following steps:

Get a connection from the pool.

Create a buffered cursor.

Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 

Returned the connection back to the pool.

The output should be as follows:

[BookingSlot]

[Guest_name]

[Assigned to: Employee Name [Employee Role]]

In [14]:
# Task 5.1: Get a connection from the pool.
try:
    connection2 = pool.get_connection()
    print('Connected...')
except:
    print('Not possible to create a connection')

Connected...


In [15]:
# confirm the connection with the pool:

print(f"Connection established with the pool {connection2.pool_name}, with ID: {connection2.connection_id}")

Connection established with the pool pool_b, with ID: 159


In [16]:
# Task 5.2: Create a buffered cursor.

if connection2.is_connected():
    cursor2 = connection2.cursor(buffered=True)
    print('Cursor created.')
else:
    print('The connection is not established')

Cursor created.


In [17]:
# Task 5.3: Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 

query_combine = """
SELECT b.bookingSlot,
CONCAT(b.guestFirstName, ' ', b.guestLastName) AS 'Guest Name',
e.name AS 'Employee Name',
e.role AS 'Employee Role'
FROM employees e  
INNER JOIN bookings b
ON b.employeeID = e.employeeID
ORDER BY b.bookingSlot ASC;
"""
cursor2.execute(query_combine)

results6 = cursor2.fetchmany(size=3)
columns6 = cursor2.column_names

df6 = pd.DataFrame(data=results6, columns=columns6)
df6.style.hide(axis='index')

bookingSlot,Guest Name,Employee Name,Employee Role
0 days 15:00:00,Vanessa McCarthy,Giorgos Dioudis,Head Chef
0 days 17:30:00,Marcos Romero,Fatma Kaya,Assistant Chef
0 days 18:00:00,Anees Java,John Millar,Receptionist


In [18]:
# Task 5.4: Return the connection.

if connection2.is_connected():
    connection2.close()
    print('Connection closed.')
else:
    print('The connection do not exist.')

Connection closed.
